# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [13]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
%pip install feedparser


  Using cached sgmllib3k-1.0.0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [feedparser]
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

 80%|████████  | 4/5 [03:42<00:56, 56.64s/it]/Users/udaykumar/projectt/llm_engineering/week8/agents/deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|██████████| 5/5 [04:25<00:00, 53.18s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Anchor Hocking Heritage Hill 2-Gallon Glass Jar with Lid for $14 + free shipping w/ $35\nDetails: You'd pay at least $8 more elsewhere. Opt for pickup to avoid the $6.99 shipping charge (or get free shipping with orders of $35 or more). Buy Now at Walmart\nFeatures: \nURL: https://www.dealnews.com/Anchor-Hocking-Heritage-Hill-2-Gallon-Glass-Jar-with-Lid-for-14-free-shipping-w-35/21787899.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Unlocked Google Pixel Fold 256GB Android Smartphone for $375 + free shipping
Details: Apply the promo code "TECH4THEM" to drop the price to the best we've seen in any condition, and $216 less than you'd pay for a refurb elsewhere. A one-year Allstate warranty is included. Buy Now at eBay
Features: Google Tensor G2 chipset split screen multitasking compatible 

In [14]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description='Discover the power of a Refurb Unlocked Apple iPhone 15 Pro, featuring impressive storage capacity with 256GB. This smartphone is equipped with advanced capabilities and a sleek design that Apple users love. It promises reliable performance and cutting-edge technology, making it perfect for both casual use and intensive tasks. With a focus on user experience, it supports the latest iOS updates and features for improved usability and functionality.', price=444.0, url='https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-15-Pro-128-GB-Phone/468789.html?iref=rss-c142')

In [15]:
from agents.scanner_agent import ScannerAgent

In [16]:
agent = ScannerAgent()
result = agent.scan()

In [17]:
result

DealSelection(deals=[Deal(product_description='The Seenda Rechargeable Karaoke Machine comes with a powerful 12" woofer and 40W output, perfect for delivering rich sound quality at your events. It includes two wireless microphones, making it great for duets or group performances. The built-in Bluetooth 5.0 allows for easy pairing with your devices, while colorful LED party lights create an exciting atmosphere. Additional features include a telescoping handle, multiple input modes including REC/AUX/USB/TF/FM radio, and adjustable echo, treble, and bass settings to customize your audio experience.', price=63.0, url='https://www.dealnews.com/Seenda-Rechargeable-Karaoke-Machine-w-2-Wireless-Mics-for-63-free-shipping/21787983.html?iref=rss-c142'), Deal(product_description='Experience stunning visuals with the Yaber T1 Pro Native 1080P Portable Projector. This projector is equipped with advanced features including an AI-powered autofocus system that provides 5-second clarity calibration and 